In [137]:
from web3 import Web3
import pandas as pd
import requests
from decimal import Decimal
import json
from moralis import evm_api
import warnings
warnings.filterwarnings('ignore')

In [2]:
# import Excel spreadsheet with Etherscan data as dataframe

df = pd.read_excel('data/madTX.xlsx')

# Subset dataframe to keep only useful columns 

clean_df = df.iloc[:, [0, 1, 2, 3, 10, 15]]

TxGoerli = list(clean_df['Txhash'])
timestamp_list = list(clean_df['UnixTimestamp'])

In [58]:
# connect to the Ethereum network using Infura (Goerli testnet)
gw3 = Web3(Web3.HTTPProvider('https://goerli.infura.io/v3/YOUR_INFURA_API'))

# connect to the Ethereum network using Infura (Mainnet)
ew3 = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/YOUR_INFURA_API'))

# Etherscan API
eth_api_key = 'YOUR_ETHERSCAN_API'

# Moralis API
moralis_API = 'YOUR_MORALIS_API'

In [4]:
# contract address to get Metacell IDs
eth_contract_addr_mID = '0x7fd4C4381b56F6F84B86f6134104893CB74c39BC'

# create abi address (metacell IDs)
eth_abi_addr_mID = f"https://api-goerli.etherscan.io/api?module=contract&action=getabi&address={eth_contract_addr_mID}&apikey={eth_api_key}"

# abi data
eth_abi_data_mID = requests.get(eth_abi_addr_mID).json()

# create contract objects
eth_contract_mID = gw3.eth.contract('0xf7Dc87cD031A3F6A3FDC5E35305c545817FD2c66', abi=eth_abi_data_mID['result'])



In [5]:
# decode input data for each transaction

MetacellID = []
for tx_hash in TxGoerli:
    tx = gw3.eth.getTransaction(tx_hash)
    eth_func_obj, eth_func_params = eth_contract_mID.decode_function_input(tx['input'])
    MetacellID.append(eth_func_params['_tokenID'])
    
# Add Metacell IDs to dataframe
clean_df['MetacellID'] = MetacellID

In [8]:
# Get Ethereum mainnet block numbers based on timestamp 
# This will be used to collect average Tx price and further calculate cost per transaction in USDT

ETHMainBlock = []

for timestamp in timestamp_list:
    # Define the params dictionary with the current timestamp
    params = {"date": str(timestamp), "chain": "eth"}

    # Call the get_date_to_block() function with the current params
    block_number = evm_api.block.get_date_to_block(api_key=Moralis_API, params=params)
    ETHMainBlock.append(block_number['block'])
    
# Add data to dataframe
clean_df['ETHMainBlock'] = ETHMainBlock

# replace block number 16475069 with next one 16475070, as block number 16475069 has 0 transactions
clean_df['ETHMainBlock'] = clean_df['ETHMainBlock'].replace(16475069, 16475070)

In [12]:
# Get average transaction cost in Ethereum mainnet
transaction_cost = []

for block_number in list(clean_df['ETHMainBlock']):
    # Get the block information
    block = ew3.eth.getBlock(block_number)

    # Get the total gas used in the block
    total_gas_used = block.gasUsed

    # Get the number of transactions in the block
    num_transactions = len(block.transactions)

    # Get the current gas price in wei
    gas_price_wei = ew3.eth.gasPrice

    # Convert the gas price to ETH and convert to a float
    gas_price_eth = float(ew3.fromWei(gas_price_wei, 'ether'))

    # Calculate the average transaction cost
    avg_tx_cost = Decimal(total_gas_used) / Decimal(num_transactions) * Decimal(gas_price_eth)
    transaction_cost.append(avg_tx_cost)

In [138]:
# Calculate average tx cost in USDT, based on average ETH price during alpha test (19th Jan 2023 to 10th Feb 2023)

averageETHprice = 1611.5

clean_df['AvgTxCostUSDT'] = clean_df['AvgTxCostETH'].apply(lambda x: x * Decimal(str(averageETHprice)))

In [141]:
# Export clean_df dataframe to Excel file
clean_df.to_excel("data/MADSummaryProcessed.xlsx")

clean_df.head()

,Txhash,Blockno,UnixTimestamp,DateTime,TxnFee(ETH),Method,MetacellID,ETHMainBlock,AvgTxCostETH,AvgTxCostUSDT
0,0x5a012c3027074f9663d8ff952eb481af7a05973e97ad...,8412883,1675221264,2023-02-01 03:14:24,0.000184,Evolve,297,16531215,0.002734375845963993563098713416,4.406446675770975626933576670
1,0xa2a9e91bf11259cff8562d6304272bc8e1f4b4bf3453...,8412891,1675221384,2023-02-01 03:16:24,0.000184,Evolve,84,16531225,0.003124476317682892815807605366,5.035093585945981772673956047
2,0x931cc355f81327cbbf270f49e233d9247b66d72fed30...,8412893,1675221432,2023-02-01 03:17:12,0.000184,Evolve,1815,16531229,0.003572206206938232018588936162,5.756610302480960897956070625
3,0xb97143aab25b8b7a01c302cf154f07e0cc1320f08820...,8412906,1675221588,2023-02-01 03:19:48,0.000184,Evolve,1814,16531242,0.002679748993353184380633645380,4.318415502788656629391119530
4,0x6505100f5ffa0195473dcd5b5ea80f1330be0b1beae4...,8412908,1675221624,2023-02-01 03:20:24,0.000184,Evolve,1763,16531245,0.003160685479600488386521554988,5.093444650376187034879485863


In [132]:
# Get summary statistics

# group the data by metacellID and calculate count, mean for AvgTxCost and AvgCostUSD
final_df = clean_df.groupby('MetacellID').agg(
    Transactions=('MetacellID', 'count'),
    AvgTxCostETH=('AvgTxCostETH', 'mean'),
    AvgTxCostUSD=('AvgTxCostUSDT', 'mean'),
    TotalTxCostETH=('AvgTxCostETH', 'sum'),
    TotalTxCostUSD=('AvgTxCostUSDT', 'sum')
)

# Remove Metacells with less than 10 transactions
final_df = final_df[final_df['Transactions'] >= 10]

# Display dataframe
final_df

,Transactions,AvgTxCostETH,AvgTxCostUSD,TotalTxCostETH,TotalTxCostUSD
MetacellID,,,,,
84,30,0.003147,5.071184,0.09440615058574321378200889946,152.1355116689251890097073416
161,38,0.002774,4.470095,0.1054071466677203240340951627,169.8636168550313021809443545
264,25,0.003161,5.093562,0.07901895864525572867122250798,127.3390518568296067536750717
266,28,0.003322,5.353071,0.09301022670700308138721660972,149.8859803383354656554995666
297,28,0.003066,4.940393,0.08583990418609644314192872230,138.3310055958944181232181359
1039,32,0.003234,5.212067,0.1034974542915240064474352602,166.7861475907909363900419218
1041,31,0.003593,5.790484,0.1113900168928818190004431743,179.5050122228790513192141753
1585,32,0.003059,4.930003,0.09789642497132618674146843099,157.7600888412921499338763764
1763,38,0.002920,4.706096,0.1109721568128656151489021219,178.8316307039329388124557695


In [134]:
# Additionaly, we may add evolution level achieved by each Metacell 

# Add column with Metacell levels achieved (hard-coded)
LevelAchieved = [48,67,43,40,42,41,44,44,53,34,47,46,44,43,49,49,31]

# Convert the list to a Pandas Series with the same index as the DataFrame
series_to_add = pd.Series(LevelAchieved, index=final_df.index)

# Add the series as a second column to the DataFrame
final_df.insert(1, 'LevelAchieved', series_to_add)

In [143]:
# Calculate averages of the "Transactions", "LevelAchieved", "AvgTxCostETH", "AvgTxCostUSD", "TotalTxCostETH", and "TotalTxCostUSD" column
transactions_avg = final_df['Transactions'].mean()
LevelAchieved_avg = final_df['LevelAchieved'].mean()
AvgTxCostETH_avg = final_df['AvgTxCostETH'].mean()
AvgTxCostUSD_avg = final_df['AvgTxCostUSD'].mean()
TotalTxCostETH_avg = final_df['TotalTxCostETH'].mean()
TotalTxCostUSD_avg = final_df['TotalTxCostUSD'].mean()

# Print the averages
print("Number of Metacells evolved:", len(final_df))
print("Average number of Transactions:", round(transactions_avg,0))
print("Average Metacell evolution level achieved:", round(LevelAchieved_avg,0))
print("Average transaction cost in ETH:", round(AvgTxCostETH_avg,9), "ETH")
print("Average transaction cost in USDT:", round(AvgTxCostUSD_avg,2), "USDT")
print("Average total cost in ETH for 17 Metacells:", round(AvgTxCostUSD_avg,2), "ETH")
print("Average total cost in USDT for 17 Metacells:", round(TotalTxCostUSD_avg,2), "USDT")

Number of Metacells evolved: 17
Average number of Transactions: 30.0
Average Metacell evolution level achieved: 45.0
Average transaction cost in ETH: 0.00323963 ETH
Average transaction cost in USDT: 5.22 USDT
Average total cost in ETH for 17 Metacells: 5.22 ETH
Average total cost in USDT for 17 Metacells: 158.66 USDT


In [136]:
# Export final_df to Excel file
final_df.to_excel("data/MADSummaryResults.xlsx")